In [ ]:
pip install cryptography stegano pillow scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 27.7 MB/s eta 0:00:00
  Attempting uninstall: pillow
    Found existing installation: pillow 11.1.0
    Uninstalling pillow-11.1.0:
      Successfully uninstalled pillow-11.1.0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import random
import base64
import getpass
from cryptography.fernet import Fernet, InvalidToken
from stegano import lsb
from PIL import Image
import shutil

# Function to create a valid encryption key from input
def create_key_from_input(user_key):
    key_bytes = user_key.encode()
    padded_key = base64.urlsafe_b64encode(key_bytes.ljust(32)[:32])
    return padded_key

# Function to encrypt a message
def encrypt_message(secret_key, message):
    cipher_suite = Fernet(secret_key)
    return cipher_suite.encrypt(message.encode())

# Function to decrypt a message
def decrypt_message(secret_key, encrypted_message):
    cipher_suite = Fernet(secret_key)
    return cipher_suite.decrypt(encrypted_message).decode()

# Function to convert any image format to PNG (required by LSB stegano)
def convert_to_png(image_path, output_path):
    with Image.open(image_path) as img:
        img = img.convert("RGB")
        img.save(output_path, "PNG")

# Function to embed a message in an image
def embed_message(image_path, encrypted_message, output_path):
    temp_image_path = os.path.splitext(output_path)[0] + "_temp.png"
    convert_to_png(image_path, temp_image_path)
    hidden_image = lsb.hide(temp_image_path, encrypted_message.decode())
    hidden_image.save(output_path)
    os.remove(temp_image_path)

# Function to extract a hidden message from an image
def extract_message(image_path):
    try:
        return lsb.reveal(image_path)
    except Exception:
        return None

# Function to encrypt the secure key with a secondary password
def encrypt_secure_key(secure_key):
    print("\nEncrypting the secure key...")
    secondary_password = getpass.getpass("Enter a secondary password to encrypt the key: ").strip()
    if len(secondary_password) < 8:
        print("The password must be at least 8 characters long.")
        return None
    encryption_key = create_key_from_input(secondary_password)
    encrypted_key = encrypt_message(encryption_key, secure_key)
    print("Secure key successfully encrypted.")
    return encrypted_key, secondary_password

# Function to decrypt the secure key with a secondary password
def decrypt_secure_key(encrypted_key):
    print("\nDecrypting the secure key...")
    secondary_password = getpass.getpass("Enter the secondary password to decrypt the key: ").strip()
    if len(secondary_password) < 8:
        print("The password must be at least 8 characters long.")
        return None
    encryption_key = create_key_from_input(secondary_password)
    try:
        decrypted_key = decrypt_message(encryption_key, encrypted_key)
        print("Secure key successfully decrypted.")
        return decrypted_key
    except InvalidToken:
        print("Incorrect password. Unable to decrypt the secure key.")
        return None

# Main function for embedding a secret message
def main():
    # Directory containing images
    images_folder = "/content/drive/MyDrive/pushpa"
    image_paths = [os.path.join(images_folder, file) for file in os.listdir(images_folder) if file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff'))]
    if len(image_paths) < 1:
        print("No images found in the 'images' directory. Add images to proceed.")
        return

    # Select a random image from the directory
    selected_image_path = random.choice(image_paths)

    # Prompt the user for a secure key
    user_key = getpass.getpass("Enter a secure key (at least 8 characters): ").strip()
    if len(user_key) < 8:
        print("The key must be at least 8 characters long.")
        return
    secure_key = create_key_from_input(user_key)

    # Encrypt the secure key with a secondary password
    encrypted_key, secondary_password = encrypt_secure_key(secure_key.decode())
    if not encrypted_key:
        return

    # Prompt the user to enter the secret message
    secret_message = input("Enter the secret message to embed: ").strip()
    if not secret_message:
        print("The secret message cannot be empty.")
        return

    # Encrypt the secret message
    encrypted_message = encrypt_message(secure_key, secret_message)

    # Embed the message into the selected image
    temp_output_path = os.path.splitext(selected_image_path)[0] + "_temp.png"
    embed_message(selected_image_path, encrypted_message, temp_output_path)

    # Replace the original image with the modified one
    os.remove(selected_image_path)
    os.rename(temp_output_path, selected_image_path)

    # Create a zip file of the images folder for download
    output_zip_path = "/content"
    shutil.make_archive(os.path.splitext(output_zip_path)[0], 'zip', images_folder)

    print("\nThe secret message has been successfully embedded.")
    print("\nShare the following encrypted key with the receiver:")
    print(encrypted_key.decode())
    print("\nShare the secondary password via a secure channel for decryption.")
    print(f"\nThe modified images folder has been zipped. Download it from: {output_zip_path}")

if __name__ == "__main__":
    main()


Enter a secure key (at least 8 characters): ··········

Encrypting the secure key...
Enter a secondary password to encrypt the key: ··········
Secure key successfully encrypted.
Enter the secret message to embed: hii

The secret message has been successfully embedded.

Share the following encrypted key with the receiver:
gAAAAABn3kPQYsPoSBUicAT9wtSiVGIEZWPFEB5rZAMARbQFXr31CADvRIZgOBGdAZ6KlAJOzFobFA-8pdZeXbhQGMEtP9YMSiYgzP5yE3clTue60jb1DCcTt2ZxmpT3twirLs-cvQ-F

Share the secondary password via a secure channel for decryption.

The modified images folder has been zipped. Download it from: /content
